<a href="https://colab.research.google.com/github/taaha3244/AI-Agents/blob/main/Policy_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [18]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29  langchain_openai qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
  Using cached opentelemetry_exporter_otlp_proto_common-1.25.0-py3-none-any.whl (17 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.25.0-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

- Import libraries

In [205]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import BaseTool
from qdrant_client import QdrantClient
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.qdrant import Qdrant
import os

In [206]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

#Creating Tools

##Rag Creation Tool
-Takes input Urls , splits , embeds and indexes into a vector store

In [207]:
class RAGCreationTool(BaseTool):
    name: str = "RAGCreationTool"
    description: str = "Tool to extract data from online PDFs and create a RAG system."

    def _run(self, urls: list) -> str:
        try:
            openai_api_key = userdata.get('openai_api_key')
            qdrant_url = userdata.get('qdrant_end')
            qdrant_api_key = userdata.get('qdrant')

            all_docs = []
            for url in urls:
                loader = PDFPlumberLoader(url)
                data = loader.load()

                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=2000,
                    chunk_overlap=200
                )
                docs = text_splitter.split_documents(data)
                all_docs.extend(docs)

            embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=openai_api_key)

            qdrant = Qdrant.from_documents(
                all_docs,
                embeddings_model,
                url=qdrant_url,
                prefer_grpc=True,
                api_key=qdrant_api_key,
                collection_name="policy-agent",
            )

            return "RAG system created successfully with the given policy documents."
        except Exception as e:
            return str(e)



# Policy Extracrion Tool

In [224]:
class PolicyExtractionTool(BaseTool):
    name: str = "PolicyExtractionTool"
    description: str = "Tool to retrieve relevant policy documents from the vector database using project overview and property details."

    def _run(self, project_overview: str, property_details: str) -> str:
        try:
            # Retrieve configuration from environment variables
            qdrant_end = userdata.get('qdrant_end')
            qdrant_api_key = userdata.get('qdrant')
            openai_api_key=userdata.get('openai_api_key')

            # Initialize Qdrant client
            qdrant_client = QdrantClient(url=qdrant_end, api_key=qdrant_api_key)

            # Initialize embeddings using OpenAI
            embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=openai_api_key)

            # Combine inputs to form the query
            query = f"Project Overview: {project_overview}\nProperty Details: {property_details}"

            # Embed the input query for vector search
            query_result = embeddings.embed_query(query)

            # Perform vector search in the "policy-agent" collection
            response = qdrant_client.search(
                collection_name="policy-agent",
                query_vector=query_result,
                limit=15  # Retrieve top 10 closest vectors
            )


            return response
        except Exception as e:
            # Log the exception or handle it as per the application's error handling policy
            return f"Error processing question: {e}"



#Initialising the tools

In [225]:
# Initialize the custom tool
policy_tool= PolicyExtractionTool()
rag_creation_tool=RAGCreationTool()

#Creating Agents

In [226]:
from crewai import Agent

rag_expert_agent = Agent(
    role='RAG Expert',
    goal='Create a RAG system from provided policy URLs and inform the policy agent.',
    verbose=True,
    memory=True,
    backstory=("An expert in creating RAG systems from policy documents."),
    tools=[rag_creation_tool],
    allow_delegation=False
)


In [227]:
policy_analyst = Agent(
    role='Policy Analyst',
    goal='Extract eligibility criteria from local and federal policy documents.',
    verbose=True,
    memory=True,
    backstory=(
        "As a seasoned policy analyst, you are adept at navigating complex policy documents "
        "and extracting relevant information that can aid in securing funding for projects."
        "Include all important figures(dates, criterias) that can have impact in the report"
    ),
    tools=[policy_tool],
    allow_delegation=False
)

In [228]:
risk_assessor = Agent(
    role='Risk Assessor',
    goal='Assess the timelines and application processes for each financing option.',
    verbose=True,
    memory=True,
    backstory=(
        "With a meticulous approach, you are responsible for evaluating the risks associated "
        "with different financing options, ensuring that the project stays on track and within budget."
    ),
    tools=[policy_tool],
    allow_delegation=False
)

In [229]:
financial_analyst = Agent(
    role='Financial Analyst',
    goal='Generate a detailed report on financing options.',
    verbose=True,
    memory=True,
    backstory=(
        "You excel at financial analysis and are responsible for compiling comprehensive reports "
        "that provide insights into various financing options, their benefits, and compliance requirements."
    ),
    tools=[policy_tool],
    allow_delegation=False
)

In [230]:
presentation_specialist = Agent(
    role='Presentation Specialist',
    goal='Present findings to the property owner.',
    verbose=True,
    memory=True,
    backstory=(
        "You are skilled in creating compelling presentations in document format that effectively communicate complex information "
        "to stakeholders, helping them make informed decisions."
    ),
    allow_delegation=False
)

#Creating Tasks

In [231]:
rag_creation_task = Task(
    description="Extract data from the provided {policy_documents} and create a RAG system.",
    expected_output="Message indicating success or failure of RAG creation.",
    tools=[RAGCreationTool()],
    agent=rag_expert_agent
)

In [232]:
policy_extraction_task = Task(
    description=(
        "Analyze the provided policy documents to identify eligibility_criteria, "
        "financing options, subsidies, grants, and incentives applicable to the {project_overview} and {property_details}."
        "Use all the details provided in the {project_overview} and {property_details} especially Figures"
    ),
    expected_output='A detailed list of eligibility criteria, financing options, subsidies, grants, and incentives.',
    agent=policy_analyst
)

In [233]:
risk_assessment_task = Task(
    description=(
        "Evaluate the application processes and timelines for each identified financing option  "
        "to determine potential risks and delays keeping in view {project_overview} and {property_details}"
    ),
    expected_output='A risk assessment report detailing the application processes and timelines for each financing option.',
    context=[policy_extraction_task],
    agent=risk_assessor
)

In [234]:
financing_task = Task(
    description=(
        "Compile a list of financing options, including LIHTC applications, federal and local subsidies, "
        "green energy grants, and loan programs. Provide an explanation of the potential value, benefits, "
        "compliance requirements, and timelines for each option."
        "Include an explanation of the potential value and benefits each option brings to the project,"
    ),
    expected_output='A detailed report on financing options.',
    agent=financial_analyst,
    context=[policy_extraction_task],
    async_execution=False,
)

In [235]:
presentation_task = Task(
    description=(
        "Create a document presentation  all the insights from the agents. Offer guidance on the next steps "
        "and application processes."
    ),
    expected_output='A document file showing the key findings and recommendations.',
    agent=presentation_specialist,
    async_execution=False,
    context=[policy_extraction_task,risk_assessment_task,financing_task],
    output_file='findings.md'
)

In [236]:
crew = Crew(
    agents=[rag_expert_agent,policy_analyst, risk_assessor, financial_analyst, presentation_specialist],
    tasks=[rag_creation_task,policy_extraction_task, risk_assessment_task, financing_task, presentation_task],
    process=Process.sequential
)

In [237]:
result = crew.kickoff(inputs={
    'policy_documents': [
        "https://dhcd.dc.gov/sites/default/files/dc/sites/dhcd/publication/attachments/2023%20Qualified%20Allocation%20Plan%2007282023_1.pdf",
        "https://dhcd.dc.gov/sites/default/files/dc/sites/dhcd/publication/attachments/2023%20Affordable%20Housing%20RFP%2007282023_3.pdf"
    ],
    'property_details': {
        'address': "2330 Good Hope Rd., S.E., Washington, D.C.",
        'ward': 8,
        'owner': "LLC (for profit)",
        'units': 257,
        'subsidies': "Section 8",
        'complaints': "Frequent housing conditions complaints, including asthma triggers and kids’ visits to the ED for asthma attacks."
    },
    'project_overview': {
        'description': (
            "The owner of Marbury Plaza wants to finance a comprehensive, deep energy retrofit of the property, "
            "including a 500kW solar project on the roof. The retrofit measures include: "
            "Envelope Improvements, Electric Lighting, Efficient Equipment, Heating, Cooling & Ventilation."
            "costs include  retrofit_measures $50000000, rooftop_solar_project :$1500000, total :$51500000"
            "Project timeline is : start : January 2025, completion: July 2026"
            "compliance standards are Domestic sourcing of materials ,Prevailing wage requirements, Davis-Bacon requirements"
            "A few assumptions include: The project will meet compliance standards related to workforce and materials,All necessary permits and approvals will be obtained in a timely manner,"
            "The owner is willing to explore and apply for various financing options."
        ),
    }
})



> Entering new CrewAgentExecutor chain...
I need to use the RAGCreationTool to extract data from the provided policy URLs and create a RAG system.

Action: RAGCreationTool
Action Input: {"urls": ['https://dhcd.dc.gov/sites/default/files/dc/sites/dhcd/publication/attachments/2023%20Qualified%20Allocation%20Plan%2007282023_1.pdf', 'https://dhcd.dc.gov/sites/default/files/dc/sites/dhcd/publication/attachments/2023%20Affordable%20Housing%20RFP%2007282023_3.pdf']} 

RAG system created successfully with the given policy documents.

Final Answer: RAG system created successfully with the data extracted from the provided policy URLs.

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to extract eligibility criteria, financing options, subsidies, grants, and incentives from the provided policy documents using the PolicyExtractionTool.

Action: PolicyExtractionTool
Action Input: {"project_overview": "comprehensive, deep energy retrofit of the property, including a 500kW solar

In [238]:
from IPython.display import Markdown
Markdown(result)

**Affordable Housing Project Financing Options for Marbury Plaza**

**Key Findings:**
After conducting a thorough analysis of the available financing options for the affordable housing project in Washington, DC, the following key findings have been identified:

1. Low Income Housing Tax Credits (LIHTC) are competitive and require compliance with net zero energy requirements for projects with at least 10,000 square feet of gross floor area.

2. Private Activity Bond Volume Cap (Bond Cap) issued by the District of Columbia Housing Finance Agency (DCHFA) is associated with LIHTCs and requires projects to meet efficiency requirements.

3. Operating Subsidies through the Local Rent Supplement Program (LRSP) are determined by unit characteristics under the administration of the DC Housing Authority (DCHA).

4. DC Green Bank offers reduced-interest construction and permanent loans for clean energy, energy, and resource efficiency, and resilience improvements.

5. HUD’s Green and Resilient Retrofit Program (GRRP) provides funding for energy efficiency and resilience measures in affordable housing projects.

6. Affordable Housing Retrofit Accelerator delivers technical assistance for buildings subject to DC Building Energy Performance Standards.

**Recommendations:**
Based on the findings, the following recommendations are proposed for the financing of Marbury Plaza:

1. Utilize a combination of LIHTCs, Bond Cap, and Operating Subsidies to secure funding for the project while ensuring compliance with energy efficiency requirements.

2. Consider leveraging the resources of DC Green Bank for reduced-interest loans to support clean energy and resilience improvements.

3. Explore the opportunities provided by HUD’s GRRP to access funding for energy efficiency and resilience measures in the project.

4. Engage with the Affordable Housing Retrofit Accelerator to receive technical assistance for meeting DC Building Energy Performance Standards.

**Next Steps:**
To proceed with the financing of Marbury Plaza, the following next steps are recommended:

1. Develop a comprehensive financing plan that includes a mix of the identified options to meet the project's financial goals and compliance requirements.

2. Initiate discussions with the relevant agencies and organizations to secure the necessary funding and technical assistance for the project.

3. Ensure timely completion of all assessments, certifications, and compliance requirements to access the identified financing options.

**Application Processes:**
The application processes for each financing option have been outlined, highlighting the risks and timelines associated with each:

1. DC Green Bank: Provides reduced-interest loans for clean energy and resilience improvements, with risks related to funding availability and project specifics.

2. HUD’s GRRP: Offers funding for energy efficiency and resilience measures, with risks related to award schedules and resource availability.

3. Affordable Housing Retrofit Accelerator: Delivers technical assistance for compliance with DC Building Energy Performance Standards, with risks related to program requirements and timelines.

By carefully considering the key findings, recommendations, next steps, and risk assessment of the application processes for each financing option, the successful implementation of the affordable housing project at Marbury Plaza can be ensured. The combination of these financing options will provide the necessary capital and support to meet the project's financial goals and compliance requirements effectively.